<img src="https://images.squarespace-cdn.com/content/v1/5e54471da3971960f69d9535/a06ff23d-9911-4787-9887-2ff08de7d8c3/logo-quantum-brilliance.png" alt="Copyright (c) 2022 Quantum Brilliance Pty Ltd" width="240">

## Example of a classical Hamiltonian with VQE in QB SDK

In [1]:
import qbos_op
from scipy.optimize import minimize, show_options, Bounds

### Function for scipy.optimize to call

In [2]:
def qbvqe(theta,m_qv):
    for el in range(len(theta)):
        m_qv.theta[0][0][el] = theta[el]
    m_qv.run()
    return m_qv.out_energy[0][0][0]

### VQE for classical 4-qubit Hamiltonian, using built-in optimiser (Nelder-Mead)

In [3]:
qv=qbos_op.vqe()
qv.sn = 0   # No shots - deterministic VQE

# Example from user guide: 3.1.5 Example 2 - 4 qubit Hamiltonian, default (pennylane) ansatz
qv.qn = 4   
qv.ham = "0.04207897647782276 + 0.17771287465139946 Z0 + 0.1777128746513994 Z1 + -0.24274280513140462 Z2 - 0.17059738328801052 Z0Z1 + 0.6622334 Z0Z1Z2Z3"
qv.ansatz = "default"
qv.ansatz_depth = 4
qv.maxeval = 400
qv.theta = qbos_op.ND()
optimum_energy = qbvqe([0.11]*qv.ansatz_depth[0][0]*qv.qn[0][0]*3,qv)
print('Min. energy: ' + str(optimum_energy))


Min. energy: -1.3791135054777526


### VQE for classical 4-qubit Hamiltonian, using SLSQP in scipy.optimize

We first need to apply value constraints on the parameters before using SLSQP:

In [4]:
qvbound = Bounds(-3.14159,3.14159,True)

Now proceed to run ```minimize()```. 

**Important**: ensure that the number of parameters provided in the initial list matches the number required by the ansatz used.  For the default ansatz, this equals ```3*qn*ansatz_depth```:

In [5]:
qv.maxeval = 1
res = minimize(qbvqe, [0.11]*qv.ansatz_depth[0][0]*qv.qn[0][0]*3, args=(qv,), method='SLSQP', bounds=qvbound , tol=1e-6)

Inspect the optimum results returned by ```minimize()```

In [6]:
res

     fun: -1.3889184262409766
     jac: array([-1.81391835e-04,  7.45058060e-08,  0.00000000e+00,  1.16080046e-04,
        8.94069672e-08,  7.45058060e-08, -3.43278050e-04, -7.00354576e-06,
        8.94069672e-08, -4.29749489e-04, -8.00192356e-06,  8.94069672e-08,
        1.41561031e-04,  4.47034836e-08,  8.94069672e-08, -1.80706382e-04,
        4.26173210e-06,  7.45058060e-08,  1.22055411e-04, -8.32974911e-06,
       -7.01844692e-06, -4.35456634e-04, -1.17272139e-05, -1.20848417e-05,
       -9.57995653e-05,  0.00000000e+00,  0.00000000e+00,  5.13792038e-05,
       -4.17232513e-06, -4.17232513e-06, -1.69113278e-04,  4.09781933e-06,
        4.32133675e-06,  4.28363681e-04, -1.40815973e-05,  3.68058681e-06,
       -9.18582082e-04,  5.96046448e-08,  5.96046448e-08, -4.52846289e-05,
        0.00000000e+00,  4.23192978e-06,  4.05743718e-04, -2.98023224e-08,
        8.21053982e-06, -4.36380506e-04,  0.00000000e+00,  9.23871994e-06])
 message: 'Optimization terminated successfully'
    nfev: 